In [1]:
import pandas as pd
import os
import csv
from datetime import datetime
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from sqlalchemy.orm import Session

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# Create a connection to the engine called `con`
con = engine.connect()

In [4]:
Base = declarative_base()

In [5]:
# Create Measurement and Station Classes
# ----------------------------------
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    date = Column(String(15))
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    name = Column(String(45))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [6]:
# Using `create_all`, creates the tables in the database
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [7]:
# Load the cleaned csv file into a pandas dataframe
clean_measurements_path = os.path.join("Resources", "clean_hawaii_measurements.csv")
stations_path = os.path.join("Resources", "hawaii_stations.csv")

In [8]:
clean_measurements_df = pd.read_csv(clean_measurements_path)
clean_measurements_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [9]:
clean_measurements_df = clean_measurements_df[['station', 'date', 'prcp', 'tobs']]
clean_measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [10]:
stations_df = pd.read_csv(stations_path)
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [11]:
# Store and commit measurement info
measurements = []

for index, row in clean_measurements_df.iterrows():
    
    measurement = Measurements(station=row['station'], date=row['date'], 
                               prcp=row['prcp'], tobs=row['tobs'])

    measurements.append(measurement)

session.bulk_save_objects(measurements)
session.commit()

In [12]:
# test connection

# measurements = (session.query(Measurements).limit(5))

# for measurement in measurements:
#     print(measurement.station, measurement.date, measurement.prcp, measurement.tobs)

In [13]:
# Store and commit station info
stations = []

for index, row in stations_df.iterrows():
    
    station = Station(station=row['station'], name=row['name'], latitude=row['latitude'],
                      longitude=row['longitude'],elevation=row['elevation'])

    stations.append(station)

session.bulk_save_objects(stations)
session.commit()

In [14]:
# test connection
stations = (session.query(Station).limit(5))

for station in stations:
    print(station.station, station.name, station.latitude, station.longitude, station.elevation)

USC00519397 WAIKIKI 717.2, HI US 21.2716 -157.8168 3.0
USC00513117 KANEOHE 838.1, HI US 21.4234 -157.8015 14.6
USC00514830 KUALOA RANCH HEADQUARTERS 886.9, HI US 21.5213 -157.8374 7.0
USC00517948 PEARL CITY, HI US 21.3934 -157.9751 11.9
USC00518838 UPPER WAHIAWA 874.3, HI US 21.4992 -158.0111 306.6
